In [13]:
# cd /content/drive/MyDrive/adsr/

### Pycharm SSH

In [1]:
!pip install colab_ssh --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="123123")

In [2]:
!nvidia-smi

Mon May 30 05:51:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:

# from google.colab import drive
# drive.mount('/content/drive')

# # https://dashboard.ngrok.com/get-started/setup
# # 에서 생성한 토큰번호
# NGROK_TOKEN = '28dz3LY5hIrxNlx2KKTZptl0mMw_4AeJP5SprScWhq1LzFUtY'
# PASSWORD = '123123'

# from colab_ssh import launch_ssh
# launch_ssh(NGROK_TOKEN,PASSWORD)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started


PermissionError: ignored

In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 346 kB 6.2 MB/s 
     |████████████████████████████████| 140 kB 43.0 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 51.8 MB/s 
     |████████████████████████████████| 127 kB 51.5 MB/s 
     |████████████████████████████████| 144 kB 37.9 MB/s 
     |████████████████████████████████| 271 kB 40.0 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
import transformers
from datasets import (
    load_metric,
    load_from_disk,
    Dataset,
    DatasetDict,
)

import pandas as pd
import numpy as np

import re
import json

In [ ]:
data_dir = os.path.join(os.getcwd(),'data')
print(f'Data Directory at {data_dir}')

Data Directory at /content/drive/MyDrive/adsr/data


In [ ]:
train_concat = load_from_disk(os.path.join(data_dir, 'train_dataset'))
# test = load_from_disk(os.path.join(data_dir, 'test_dataset'))

train = train_concat['train']
test  = train_concat['validation']

In [ ]:
# test['context']
# test['question']

In [ ]:
print(f'Train set size : {train.shape}')
print(f'Test set size : {test.shape}')

Train set size : (4792, 7)
Test set size : (4792, 7)


In [ ]:
# 

In [ ]:
# class DatasetProcessor():
#     def __init__(self, data: pd.DataFrame):
#         self.data = data

#     def customize_dataset(self):
#         """ customize MRC dataset suitable for passage retrieval """
        
#         contexts = []
#         questions = []
#         total = []
#         for ind in range(len(self.data)):
#             if self.data['data'][ind]['source'] == 'wikipedia':
#                 if len(self.data['data'][ind]['paragraphs']) == 1:
#                     for j in range(len(self.data['data'][ind]['paragraphs'])):
#                         if self.data['data'][ind]['paragraphs'][0]['qas'][j]['question']:
#                             contexts.append(self.data['data'][ind]['paragraphs'][0]['context'])
#                             questions.append(self.data['data'][ind]['paragraphs'][0]['qas'][0]['question'])

#             # print(contexts,questions)

#         # data_frame = pd.DataFrame()
#         return contexts, questions


#     def split_train_test(data: pd.DataFrame):
#         """ split train -> train/dev set """

# data_gen = DatasetProcessor(data = train)

In [ ]:
def preprocess(context):
    """   """
    context = re.sub(r'\n', " ", context)
    context = re.sub(r"\\n", " ", context)
    context = re.sub(r"\*", " ", context)
    return context

In [ ]:
# wiki = pd.read_json(os.path.join(data_dir, 'wikipedia_documents,json'))

with open(os.path.join(data_dir, "wikipedia_documents.json" ), "r", encoding="utf-8") as f:
            wiki = json.load(f)

wiki_contexts = [preprocess(v['text']) for v in wiki.values()]

In [ ]:
import os
import json
import time
import pickle
import numpy as np
import pandas as pd
import torch

from contextlib import contextmanager
from typing import List, Tuple, NoReturn, Any, Optional, Union

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, HfArgumentParser
# from arguments import RetrieverArguments


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")


class SparseRetrieval_TFIDF():
    def __init__(self, tokenize_fn,data_path: Optional[str] = "../data/",
        context_path: Optional[str] = "wikipedia_documents.json") -> NoReturn:

        """Passage 파일을 불러오고 TfidfVectorizer를 선언"""

        self.data_path = data_path
        with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
            wiki = json.load(f)

        self.contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))  # set 은 매번 순서가 바뀌므로

        print(f"Lengths of unique contexts : {len(self.contexts)}")
        self.ids = list(range(len(self.contexts)))

        # Transform by vectorizer
        self.tfidfv = TfidfVectorizer(
            tokenizer=tokenize_fn,
            ngram_range=(1, 2),
            max_features=50000,
        )

        self.p_embedding = None 

    def get_sparse_embedding(self) -> NoReturn:
        """Create or import embeddings"""

        pickle_name = f"sparse_embedding.bin"
        tfidfv_name = f"tfidv.bin"
        emd_path = os.path.join(self.data_path, pickle_name)
        tfidfv_path = os.path.join(self.data_path, tfidfv_name)

        if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
            with open(emd_path, "rb") as file:
                self.p_embedding = pickle.load(file)
            with open(tfidfv_path, "rb") as file:
                self.tfidfv = pickle.load(file)
            print("Embedding pickle load.")
        else:
            print("Build passage embedding")
            self.p_embedding = self.tfidfv.fit_transform(self.contexts)
            print(self.p_embedding.shape)
            with open(emd_path, "wb") as file:
                pickle.dump(self.p_embedding, file)
            with open(tfidfv_path, "wb") as file:
                pickle.dump(self.tfidfv, file)
            print("Embedding pickle saved.")


    def retrieve(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1
    ) -> Union[Tuple[List, List], pd.DataFrame]:

        """Query들에 대해서 retrieved 된 Passage 반환"""

        assert self.p_embedding is not None, "get_sparse_embedding() 메소드를 먼저 수행해줘야합니다."


        if isinstance(query_or_dataset, Dataset):

            total = []
            with timer("query exhaustive search"):
                doc_scores, doc_indices = self.get_relevant_doc_bulk(
                    query_or_dataset["question"], k=topk
                )
            for idx, example in enumerate(
                tqdm(query_or_dataset, desc="Sparse retrieval: ")
            ):
                tmp = {
                    "question": example["question"],
                    "id": example["id"],
                    "context_id": doc_indices[idx],
                    "context": " ".join(
                        [self.contexts[pid] for pid in doc_indices[idx]]
                    ),
                }
                if "context" in example.keys() and "answers" in example.keys():
                    # if validation set
                    tmp["original_context"] = example["context"]
                    tmp["answers"] = example["answers"]
                total.append(tmp)

            cqas = pd.DataFrame(total)
            return cqas


    def get_relevant_doc_bulk( self, queries: List, k: Optional[int] = 1
    ) -> Tuple[List, List]:

        """top k 개의 score&indice들에 대한 pickle 파일 저장 혹은 불러오기 작업 수행"""

        query_vec = self.tfidfv.transform(queries)
        assert (
            np.sum(query_vec) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        result = query_vec * self.p_embedding.T
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        doc_scores = []
        doc_indices = []
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        return doc_scores, doc_indices


# measuring topk retrieval performance
# Test
org_dataset = load_from_disk(os.path.join(os.getcwd(),"data/train_dataset"))
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)


tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
retriever = SparseRetrieval_TFIDF(
tokenize_fn=tokenizer.tokenize, data_path="/opt/ml/data/", context_path="wikipedia_documents.json"
    ) 
retriever.get_sparse_embedding_bm25()
df =  retriever.retrieve(full_ds,topk = 10)

def retriever_prec_k(topk_list):
        result_dict = {}
        with timer("bulk query by exhaustive search"):
            for k in tqdm(topk_list):
                result_retriever = retriever.retrieve(full_ds,topk = k)
                correct = 0
                for index in range(len(result_retriever)):
                    if result_retriever['original_context'][index] in result_retriever['context'][index]:
                        correct += 1
                result_dict[k] = correct/len(result_retriever)
                print(result_dict)
        return result_dict

result = retriever_prec_k([1,3,5,10])

print(result)

Flattening the indices:   0%|          | 0/5 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/5 [00:00<?, ?ba/s]

**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 9584
})


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

NameError: ignored

'/root'